# Spin-1/2 XY model
We consider the Hamiltonian
$$
H = \sum_i (S_i^x S_{i+1}^x + S_i^y S_{i+1}^y).
$$
First, without considering any symmetry, we can construct the many-body Hamiltonian canonically, by creating:
1. a list of local operator `mats`,
2. a list of indices `inds` that specifies where the local operators act on,
3. a `basis` object specify the Hilbert space.
Without any symmetry, the basis is the tensor basis.

In [1]:
DEV = true
if DEV    # use local package
    include("../src/EDKit.jl")
    using .EDKit
else      # use EDKit in the Pkg system
    using EDKit
end

## Construct Hamiltonian

In [2]:
L = 10
# the `spin` function helps create local operators.
mat = spin((1, "xx"), (1, "yy"))
mats = fill(mat, L)
inds = [[i,mod(i,L)+1] for i in 1:L]
basis = TensorBasis(L, base=2)
H = operator(mats, inds, basis)

Operator of size (1024, 1024) with 10 terms.


For translational invariant system, the function `trans_inv_operator`, the following constructions all work the same:

In [3]:
H2 = trans_inv_operator(mat, 1:2, basis)
H3 = trans_inv_operator(mat, 2, L)

Operator of size (1024, 1024) with 10 terms.


## Operator object
The `Operator` object works like a matrix. It can act on a vector:

In [4]:
vec = rand(1024)
vec2 = H * vec
typeof(vec2)

Vector{Float64} (alias for Array{Float64, 1})

It can also be converted to ordinary matrix:

In [5]:
H_mat = Array(H) 
typeof(H_mat)

Matrix{Float64} (alias for Array{Float64, 2})

We can obtain its eigen system by:

In [6]:
using LinearAlgebra
vals, vecs = eigen(Hermitian(H));

## Symmetries
We can reduce the dimension of the many-body Hilbert space by imposing symmetry constraints. This is done by construct symmetric basis.

For example, since XY model conserve total $S^z$, we can create the basis with $\sum_i S_i^z = L/2$, by:

In [7]:
basis2 = ProjectedBasis(L=L, N=L÷2)
size(basis, 1)

1024

We see the Hilbert space is now reduced to 252. The Hamiltonian can be constructed similarly:

In [8]:
H2 = trans_inv_operator(mat, 2, basis2)

Operator of size (252, 252) with 10 terms.


The translational symmetry can further reduce the dimension. If we consider the half-filled, $k=0$ subspace.

In [9]:
basis3 = TranslationalBasis(L=L, N=L÷2, k=0)
H3 = trans_inv_operator(mat, 2, basis3)

Operator of size (26, 26) with 10 terms.


We can also impose spatio reflection symmetry (choosing $p=+1$) by:

In [10]:
basis4 = TranslationParityBasis(L=L, N=L÷2, k=0, p=1)
H4 = trans_inv_operator(mat, 2, basis4)

Operator of size (16, 16) with 10 terms.


## Entanglement

The basis object also helps to calculate the entanglement entropy of a many-body state in a specific symmetry sector. Consider now an eigenstate in the half-filled, $k=0$, and $p=+1$ sector:

In [11]:
vals, vecs = eigen(Hermitian(H4))
vec = vecs[:,1];

The bipartite entanglement entropy can be calculated using the `ent_S` function:

In [12]:
S = ent_S(vec, 1:L÷2,  basis4)

1.6602233536175888